In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime

In [3]:
import xml.etree.ElementTree as ET

In [4]:
from dateutil import parser

In [85]:
class Article:
    
    def __init__(self, company, timestamp, headline, article_text):
        self.company = company
        self.timestamp = parser.parse(timestamp)
        self.headline = headline
        self.article_text = article_text
    
    def __repr__(self):
        return self.company + "; " + str(self.timestamp) + "; " + self.headline
    
    def __eq__(self, other):
        if isinstance(other, Article):
            return (self.company == other.company and self.timestamp == other.timestamp and
                self.headline == other.headline)
        return False
    
    def __lt__(self, other):
        """
        Comparison operator for sorting Articles by timestamp. 
        """
        return self.timestamp < other.timestamp
    
    def __hash__(self):
        return hash(self.__repr__())
    
    def elapsed_time_between(self, other):
        """
        Returns the amount of time, in seconds, between the publishing 
        of this Article and another Article, other. 
        """
        elapsed_time = self.timestamp - other.timestamp
        return abs(elapsed_time.total_seconds())

~~Parsing works, but is slow for the large file. Need to implement filtering out articles that are older than 72 hours whenever a new article is added, and also put some stuff in functions.~~

In [73]:
def create_article_map(filename, k_hours=72):
    """
    Main parsing function. Takes in a file with .nml extension
    and returns a dictionary with keys that correspond to company symbols,
    and values that are sets of Article objects whose articles are about
    that company. 
    
    Arguments:
        filename: .nml file to parse
        k_hours: Argument passed in to filter_old_articles(). Determines
            article filtering. Default is 72, so articles mapped to any
            given company which are at least 72 hours older than the
            current article being parsed will be filtered from the map.
    
    Returns:
        company_article_map: Map from companies to sets of Articles about
            those companies. The set for each company will contain articles that
            were published within k_hours hours of each other. 
    """
    company_article_map = {}
    curr_file_str = ""
    with open(filename) as myfile:
        while True:
            next_line = next(myfile, "EOF")
            if next_line == "EOF":
                break
            curr_file_str += next_line
            if next_line == "</doc>\n":

                xml_elem = ET.fromstring(curr_file_str)
                company = xml_elem.find(".//djn-company-sig")
                if company is None:
                    curr_file_str = ""
                    continue
                if company[0].attrib.get('about', 'no about value') != 'Y':
                    curr_file_str = ""
                    continue
                company = company[0].text
                timestamp = xml_elem.find(".//djn-mdata").attrib['display-date']
                headline = xml_elem.find(".//headline").text
                all_text = xml_elem.find(".//text")
                article_text = "".join(all_text.itertext())

                new_article = Article(company, timestamp, headline, article_text)
                company_articles = filter_old_articles(company_article_map, new_article, k_hours)
                company_article_map[company] = company_articles

                curr_file_str = ""
    return company_article_map

In [87]:
full_sample = create_article_map("2001_sample_10M.nml", 72)

In [179]:
sorted(full_sample['AAPL'], reverse=True)

[AAPL; 2001-01-26 13:00:00+00:00; 
 SMARTMONEY.COM: Strategies For Survival In PCs,
 AAPL; 2001-01-26 01:08:00+00:00; 
 SMARTMONEY.COM: Strategies For Survival In PCs,
 AAPL; 2001-01-25 04:49:00+00:00; 
 WSJ(1/25): Latest Apple Laptop Offers A New Look And Feel]

In [86]:
def filter_old_articles(company_article_map, curr_article, k_hours=72):
    """
    Check the set of articles mapped to curr_article.company, and 
    filter out articles that are at least k_hours older than curr_article. 
    
    Arguments:
        curr_article: An Article object, used to compare time stamps with all other
            articles in company_article_map[curr_article.company]. 
        k_hours: An int, default 72. All articles in company_article_map[curr_article.company] 
            that have at least k_hours elapsed time between that article and 
            curr_article.timestamp will be filtered from the map.
            
    Returns:
        company_article_set: A new set to map to curr_article.company, with 
            old articles filtered out, with "old" specified as k_hours.
    """
    
    k_seconds = k_hours * 60 * 60
    curr_company = curr_article.company
    curr_timestamp = curr_article.timestamp
    
    company_article_set = company_article_map.get(curr_company, "No articles for this company")
    
    if company_article_set == "No articles for this company":
        return set([curr_article])
    
    articles_to_remove = set()
    for article in company_article_set:
        if curr_article.elapsed_time_between(article) >= k_seconds:
            articles_to_remove.add(article)
            
    company_article_set.difference_update(articles_to_remove)
    company_article_set.add(curr_article)
    
    return company_article_set  

In [115]:
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

In [116]:
stop_words = set(stopwords.words('english'))
ps = PorterStemmer()

In [139]:
def stem_and_filter(article):
    """
    Takes an Article object and tokenizes the article text into a list, then
    removes stop words from the list and stems all the remaining words. Returns
    a list of stemmed words in the article.
    """
    text = article.article_text
    tokenized = word_tokenize(text)
    no_stop_words = [w for w in tokenized if w not in stop_words]
    stemmed = [ps.stem(w) for w in no_stop_words]
    return stemmed

In [195]:
def bow_similarity_score(s1, s2):
    """
    Returns the bag-of-words similarity score between an article s1 and article s2.
    Specifically, measures percentage of words in s1 that are also in s2. 
    s1, s2 must be tokenized and stemmed articles.
    """
    return len(set(s1).intersection(s2)) / len(set(s1))

In [202]:
def articles_older_than(article, article_set):
    """
    Takes in a list or set of articles pertaining to the same company
    as article, and returns a new list containing all articles in 
    article_set which were published earlier than article.
    """
    return list(filter(lambda x: x < article, article_set))

In [230]:
company_article_map = full_sample
def old_and_closest_neighbor_score(curr_article, article_set, num_closest=5):
    """
    Calculates Old(s) and ClosestNeighbor(s), where s is curr_article. 
    
    Arguments:
        curr_article: An Article object for which to calculate the scores
        article_set: A set of Article objects with the same company as curr_article.company
        num_closest: The number of closest (by similarity score) stories to look at 
            to calculate Old(s)
    
    Returns:
        old_score: Old(s)
        closest_neighbor_score: ClosestNeighbor(s)
    """
    
    article_list = articles_older_than(curr_article, article_set)

    sim_scores = []
    stemmed_articles = []
    
    curr_article_stemmed = stem_and_filter(curr_article)
    
    for article in article_list:
        stemmed_and_filtered = stem_and_filter(article)
        stemmed_articles.append(stemmed_and_filtered)
        sim_scores.append(bow_similarity_score(curr_article_stemmed, stemmed_and_filtered))
    
    curr_article_stemmed_unique = set(curr_article_stemmed)
    closest_articles_indices = np.argsort(sim_scores)[::-1][:num_closest]
    closest_articles = [stemmed_articles[i] for i in closest_articles_indices]
    closest_articles_union = set().union(*closest_articles)
    intersect_with_closest_n = curr_article_stemmed_unique.intersection(closest_articles_union)
    
    old_score = len(intersect_with_closest_n) / len(curr_article_stemmed_unique)
    closest_neighbor_score = bow_similarity_score(curr_article_stemmed, closest_articles[0])
    
    return old_score, closest_neighbor_score

In [231]:
def is_old_news(article, article_set, num_closest=5):
    old, closest_neighbor = old_and_closest_neighbor_score(article, article_set, num_closest)
    return old > 0.6

In [232]:
def is_reprint(article, article_set, num_closest=5):
    old, closest_neighbor = old_and_closest_neighbor_score(article, article_set, num_closest)
    reprint = (closest_neighbor / old) >= 0.8
    return (old > 0.6) * reprint

In [233]:
def is_recombination(article, article_set, num_closest=5):
    old, closest_neighbor = old_and_closest_neighbor_score(article, article_set, num_closest)
    recomb = (closest_neighbor / old) < 0.8
    return (old > 0.6) * recomb

In [250]:
def precomputed_is_reprint(old, closest_neighbor):
    reprint = (closest_neighbor / old) >= 0.8
    return (old > 0.6) * reprint

In [251]:
def precomputed_is_recombination(old, closest_neighbor):
    reprint = (closest_neighbor / old) < 0.8
    return (old > 0.6) * reprint

In [261]:
# articles_df = pd.DataFrame(columns=['company', 'time', 'headline', 'old_score', 'is_reprint', 'is_recombination'])
iterations = 0
rows_list = []
for company in first_20_pairs:
    article_set = company_article_map[company]
    for article in article_set:
        row_dict = {}
        if len(articles_older_than(article, article_set)) > 0:
            row_dict['company'] = company
            row_dict['time'] = article.timestamp
            row_dict['headline'] = article.headline
            old, closest_neighbor = old_and_closest_neighbor_score(article, article_set)
            row_dict['old_score'] = old
            row_dict['is_reprint'] = precomputed_is_reprint(old, closest_neighbor)
            row_dict['is_recombination'] = precomputed_is_recombination(old, closest_neighbor)
        else:
            continue
        rows_list.append(row_dict)
    iterations += 1

rows_list

[{'company': 'F.BNP',
  'headline': "\nFirstMark's French Unit Sees Breakeven, EUR200M Rev In 04",
  'is_recombination': 0,
  'is_reprint': 0,
  'old_score': 0.24489795918367346,
  'time': datetime.datetime(2001, 1, 25, 12, 58, tzinfo=tzutc())},
 {'company': 'F.BNP',
  'headline': '\nCarrefour/Klepierre -2: Debt Financing For Purchase>F.CAR',
  'is_recombination': 0,
  'is_reprint': 0,
  'old_score': 0.3611111111111111,
  'time': datetime.datetime(2001, 1, 25, 16, 43, tzinfo=tzutc())},
 {'company': 'F.BNP',
  'headline': '\nBusiness News From The Greek Press Thursday',
  'is_recombination': 0,
  'is_reprint': 0,
  'old_score': 0.23529411764705882,
  'time': datetime.datetime(2001, 1, 25, 9, 49, tzinfo=tzutc())},
 {'company': 'MTA',
  'headline': '\nBusiness News From The Hungarian Press Thursday',
  'is_recombination': 0,
  'is_reprint': 0,
  'old_score': 0.29464285714285715,
  'time': datetime.datetime(2001, 1, 25, 9, 15, tzinfo=tzutc())},
 {'company': 'C.BZQ',
  'headline': '\nIsrael

In [263]:
articles_df = pd.DataFrame(rows_list)

In [264]:
articles_df

,company,headline,is_recombination,is_reprint,old_score,time
0,F.BNP,"\nFirstMark's French Unit Sees Breakeven, EUR2...",0,0,0.244898,2001-01-25 12:58:00+00:00
1,F.BNP,\nCarrefour/Klepierre -2: Debt Financing For P...,0,0,0.361111,2001-01-25 16:43:00+00:00
2,F.BNP,\nBusiness News From The Greek Press Thursday,0,0,0.235294,2001-01-25 09:49:00+00:00
3,MTA,\nBusiness News From The Hungarian Press Thursday,0,0,0.294643,2001-01-25 09:15:00+00:00
4,C.BZQ,\nIsrael/Strike -2: Most International Flights...,0,0,0.382609,2001-01-28 13:31:00+00:00
5,DJDAY,\nTreasurys Mixed As Players Mop Up After Gree...,1,0,0.625000,2001-01-26 15:32:00+00:00
6,DJDAY,"\nCharting Money: Where There's Smoke, Don't B...",1,0,0.644550,2001-01-26 17:00:00+00:00
7,DJDAY,\nN.Y. Stocks End Mixed; Spunky Show Of It By ...,0,0,0.557078,2001-01-26 21:31:00+00:00
8,DJDAY,\nCalendar Of Earnings For Next Week-2: List F...,0,0,0.281596,2001-01-26 20:31:00+00:00
9,DJDAY,\nComprehensive Stock Split Table: Pay And Rec...,0,0,0.315789,2001-01-26 14:01:00+00:00


In [258]:
first_20_pairs = {k: full_sample[k] for k in list(full_sample)[:20]}
first_20_pairs

{'A.WPL': {A.WPL; 2001-01-22 05:09:31+00:00; 
  Alcoa Australia Plans Statement Mon On Wagerup Expansion,
  A.WPL; 2001-01-22 06:27:12+00:00; 
  Shell Extends Bid For Australia's Woodside Until Feb 28,
  A.WPL; 2001-01-22 06:40:52+00:00; 
  Shell/Woodside Extends Bid -2: Follows Government Delay,
  A.WPL; 2001-01-22 06:47:47+00:00; 
  Shell/Woodside Extends Bid -3: Was Due To Close On Feb 2,
  A.WPL; 2001-01-23 03:56:23+00:00; 
  Alcoa Australia: No Comment On Wagerup Refinery Forecast,
  A.WPL; 2001-01-24 01:42:48+00:00; 
  Costello:Will Approve Shell Bid If In National Interest,
  A.WPL; 2001-01-24 01:51:36+00:00; 
  Costello/Woodside Offer -2: Conditions Sometimes Imposed,
  A.WPL; 2001-01-24 01:57:48+00:00; 
  MARKET TALK/AU-EQ: Costello Mulls Shell National Interest},
 'AOL': {AOL; 2001-01-26 13:16:00+00:00; 
  New FCC Chief Of Staff Served Briefly As Disney Lobbyist,
  AOL; 2001-01-26 15:16:00+00:00; 
  AOL Time Warner (AOL) Block: 300,000 Shrs At 54.00,
  AOL; 2001-01-26 15:36:0

In [249]:
aapl_stories = sorted(full_sample['GS'], reverse=True)
[is_reprint(aapl_stories[i], aapl_stories) for i in range(len(aapl_stories)) 
    if len(articles_older_than(aapl_stories[i], aapl_stories)) > 0]

[1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0]

In [237]:
aapl_stories[0] > aapl_stories[1]

True

In [199]:
print(amazon_stories[0].article_text, "\n")
print(amazon_stories[1].article_text)
stf_az1 = stem_and_filter(amazon_stories[0])
stf_az2 = stem_and_filter(amazon_stories[1])
# print(stf_az1)
# print(stf_az2)
print(bow_similarity_score(stf_az1, stf_az2))
print(len(set(stf_az1).intersection(stf_az2)) / len(set(stf_az1)))
print(len(set(stf_az1)))



 
 
   By Christiane Bird 
   Of DOW JONES NEWSWIRES 
 

  NEW YORK (Dow Jones)--Most mutual fund categories posted modest gains in the last week as blue chip stocks edged up, consumer cyclicals advanced and investors digested fourth-quarter earnings reports. 

  The major exceptions were telecommunications funds and science and technology funds, down 4.33% and 1.55% respectively, in the week ended Thursday, according to Lipper Inc. 

  Technology and telecommunications stocks had been market standouts so far this year, but they retreated on Thursday. JDS Uniphase Corp. (JDSU) dipped more than 12%, to $55.19, at the market's close Thursday, while Amazon.com Inc. (AMZN), Broadvision Inc. (BVSN), and Yahoo! Inc. (YHOO) all also lost ground. 

  Among U.S. diversified funds, value funds gained the most in the last week. Large-cap value funds were up 1.01%, while mid-cap value funds returned 1.97% and small-cap value funds rose 1.19%. 

  In comparison, large-cap growth funds gained 0.08

In [114]:
A = ['a', 'c', 'd']
B = ['c', 'd', 2]
C = [1, 2, 3]
set().union(*[A, B, C])

{1, 2, 3, 'd', 'a', 'c'}

In [104]:
newest_amzn_text = sorted(full_sample['AMZN'], reverse=True)[0].article_text
stop_words = set(stopwords.words('english'))
tokenized_article = word_tokenize(newest_amzn_text)
tokenized_filtered = [w for w in tokenized_article if w not in stop_words]
ps = PorterStemmer()
for w in tokenized_filtered:
    print(ps.stem(w))

By
christian
bird
Of
dow
jone
newswir
new
york
(
dow
jone
)
--
most
mutual
fund
categori
post
modest
gain
last
week
blue
chip
stock
edg
,
consum
cyclic
advanc
investor
digest
fourth-quart
earn
report
.
the
major
except
telecommun
fund
scienc
technolog
fund
,
4.33
%
1.55
%
respect
,
week
end
thursday
,
accord
lipper
inc.
technolog
telecommun
stock
market
standout
far
year
,
retreat
thursday
.
jd
uniphas
corp.
(
jdsu
)
dip
12
%
,
$
55.19
,
market
's
close
thursday
,
amazon.com
inc.
(
amzn
)
,
broadvis
inc.
(
bvsn
)
,
yahoo
!
inc.
(
yhoo
)
also
lost
ground
.
among
u.s.
diversifi
fund
,
valu
fund
gain
last
week
.
large-cap
valu
fund
1.01
%
,
mid-cap
valu
fund
return
1.97
%
small-cap
valu
fund
rose
1.19
%
.
In
comparison
,
large-cap
growth
fund
gain
0.08
%
,
mid-cap
growth
fund
rose
1.34
%
,
small-cap
growth
fund
1.06
%
.
large-cap
,
mid-cap
small-cap
core
fund
increas
0.67
%
,
1.7
%
1.36
%
respect
.
As
whole
,
5,105
u.s.
diversifi
fund
follow
lipper
rose
averag
0.93
%
week
.
dure
period
,
